In [1]:
import pandas as pd
from keras.models import load_model
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import random
from scipy.integrate import quad
from scipy.interpolate import interp1d

ann_arr = load_model('IP_Arrival_time/Arrival_ANN_1')
ann_end = load_model('IP_End_time/End_ANN_2')
ann_chg = load_model('IP_Change_time/Change_ANN_3')

#Section1
ann1 = load_model('IP_Section_1_new/Section1_ANN_3')
sc1 = pickle.load(open('IP_Section_1_new/Section1_scaler_ANN_3.pkl', 'rb'))
#Section2
ann2 = load_model('IP_Section_2_new/Section2_new_ANN_5')
sc2 = pickle.load(open('IP_Section_2_new/Section2_new_scaler_ANN_5.pkl', 'rb'))
#Section3
ann3 = load_model('IP_Section_3_new/Section3_new_ANN_1')
sc3 = pickle.load(open('IP_Section_3_new/Section3_new_scaler_ANN_1.pkl', 'rb'))
#Section4
ann4 = load_model('IP_Section_4_new/Section4_new_ANN_1')
sc4 = pickle.load(open('IP_Section_4_new/Section4_new_scaler_ANN_1.pkl', 'rb'))
#Section5
ann5 = load_model('IP_Section_5_new/Section5_new_ANN_1')
sc5 = pickle.load(open('IP_Section_5_new/Section5_new_scaler_ANN_1.pkl', 'rb'))
#Section6
ann6 = load_model('IP_Section_6_new/Section6_new_ANN_1')
sc6 = pickle.load(open('IP_Section_6_new/Section6_new_scaler_ANN_1.pkl', 'rb'))

D:\Anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
D:\Anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
D:\Anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at you

In [15]:
# Perform trapezoidal rule integration
def integrate_with_TR(time, y):
    cumulative_integration = np.cumsum(np.gradient(time) * y)
    
    return cumulative_integration

# Function to perform integration
def perform_integration(file_path):
    # Load the Excel file
    df = pd.read_excel(file_path)
    df_sorted = df.sort_values(by='Time', ascending=True)
    time_num = df_sorted['Time'].values  # Assuming time is in seconds and converting to milliseconds
    y_num = df_sorted['Pressure'].values  # Pressure units adjustment (kPa)
    max_time = np.max(time_num)
    cumulative_integration = integrate_with_TR(time_num, y_num)
    
    return max_time, cumulative_integration[-1]

# Define the function to generate time_ann and y_ann
def integrate_ann(Distance, Mass, termination_time, interval=0.01):

    arr = np.array ([[Mass,Distance]])
    arrival_time = ann_arr.predict(arr)
    end_time = ann_end.predict(arr)
    change_time = ann_chg.predict(arr)
    #print ("1",arrival_time,end_time, change_time)
    
    def stable_state(start, end, interval):    
        t = np.arange(start, end + interval, interval)
        t = t.reshape(len(t),1)
    
        p = np.repeat(0, len(t)).reshape(len(t),1)
        
        values = np.concatenate ((t, p), axis = 1)
        return values
    
    def p_fluctuation(d_set):
        d_set[:,0:3] = sc2.transform(d_set[:,0:3])
        p_f = ann2.predict(d_set)
        return p_f
    
    if termination_time < end_time:
        end_time = termination_time
        
    else:
        start = end_time
        end = termination_time
        values3 = stable_state(start, end, interval)
    #print ("2",arrival_time,end_time, Distance, Mass, Angle)
    t2= np.arange(arrival_time, end_time + interval, interval)
    t2 = t2.reshape(len(t2),1)
    
    if t2[0] < change_time:
        ph2 = np.array([[0, 1]])
    else:
        ph2 = np.array([[1, 0]])
        
    positive = np.array([0, 1])
    negative= np.array([1, 0])
    
    for time in t2[1:]:
        if time < change_time:
            ph2=np.vstack((ph2,positive))
        else:
            ph2=np.vstack((ph2,negative))
            
    m2 = np.repeat(Mass, len(t2)).reshape(len(t2),1)
    d2 = np.repeat(Distance, len(t2)).reshape(len(t2),1)
    d_set = np.concatenate ((m2, d2, t2, ph2), axis = 1)
    
    #p2 = p_fluctuation(Standoff_distance, d_set)
    p2 = p_fluctuation(d_set)
    p2 = p2.reshape((-1, 1))
    values2 = np.concatenate ((t2, p2), axis = 1)
    
    if 'values3' in locals():
        ml_model = np.concatenate((values2, values3), axis=0)
    else:
        ml_model = values2
        
    time = ml_model[:,0]
    y = ml_model[:,1]
    max_time_ann = max(time)
    
    cumulative_integration = integrate_with_TR(time, y)
    
    return cumulative_integration[-1], max_time_ann


In [16]:
# Define the root directory
root_dir = Path("G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/Far_field_time_history_dataset")

#distance_mapping = {'5': 1.5, '7': 1.75, '9': 2, '11': 2.25, '13': 2.5, '15': 2.75, '17': 3}

# List to store file paths and corresponding metadata
file_data = []

# Traverse through all directories and subdirectories
for file_path in root_dir.rglob('*.xlsx'):
    # Extract the parent directory name
    parent_dir = file_path.parent.name
    
    # Parse distance and mass from the directory name
    _, rest = parent_dir.split('_', 1)
    mass = int(rest) / 2
    file_name = file_path.stem[1:]
    #file_name = file_path.stem
    #print(file_name)
    
    if 2 < int(file_name) < 14: #Section2
    #if 12 < int(file_name) < 26: #Section3
    #if 24 < int(file_name) < 38: #Section4
        distance = np.ceil(int(file_name)/2)
        #distance = distance_mapping.get(file_name, None)
    else:
        continue
    # Append the data
    file_data.append({
        'path': str(file_path),
        'distance': float(distance),
        'mass': float(mass),
        'file name': file_name
    })

# Convert the data to a DataFrame
sampled_files = pd.DataFrame(file_data)

# Apply integration to the sampled files and add a new column
sampled_files[['termination_time', 'num_integration']] = sampled_files['path'].apply(lambda x: pd.Series(perform_integration(x)))
# Apply ANN-based model to sampled files
sampled_files[['ml_integration', 'termination_time_ann']] = sampled_files.apply(
    lambda row: pd.Series(integrate_ann(row['distance'], row['mass'], row['termination_time'])), 
    axis=1)

# Print the updated sampled files DataFrame
print(sampled_files)

# Optionally, save the updated DataFrame to a CSV for later use
sampled_files.to_excel('IP_TR_integration.xlsx', index=False)

74/74 [==============================] - 0s 2ms/step
                                                 path  distance  mass  \
0   G:\Chamodi\LSDYNA_1D\Incident_pressure_time_hi...       7.0   0.5   
1   G:\Chamodi\LSDYNA_1D\Incident_pressure_time_hi...       3.0   0.5   
2   G:\Chamodi\LSDYNA_1D\Incident_pressure_time_hi...       5.0   0.5   
3   G:\Chamodi\LSDYNA_1D\Incident_pressure_time_hi...       7.0   6.5   
4   G:\Chamodi\LSDYNA_1D\Incident_pressure_time_hi...       3.0   6.5   
5   G:\Chamodi\LSDYNA_1D\Incident_pressure_time_hi...       5.0   6.5   
6   G:\Chamodi\LSDYNA_1D\Incident_pressure_time_hi...       7.0   8.5   
7   G:\Chamodi\LSDYNA_1D\Incident_pressure_time_hi...       3.0   8.5   
8   G:\Chamodi\LSDYNA_1D\Incident_pressure_time_hi...       5.0   8.5   
9   G:\Chamodi\LSDYNA_1D\Incident_pressure_time_hi...       7.0  10.5   
10  G:\Chamodi\LSDYNA_1D\Incident_pressure_time_hi...       3.0  10.5   
11  G:\Chamodi\LSDYNA_1D\Incident_pressure_time_hi...       5.0  10.5  

In [17]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

file_path = 'IP_TR_integration.xlsx'
data = pd.read_excel(file_path, sheet_name= 'Sheet2')

#data = data.sample(frac=0.3, random_state=40)

# Extract the numerical and ML model integration values
numerical_values = data['num_integration']
ml_model_values = data['ml_integration']

# Calculate MAE
mae = mean_absolute_error(numerical_values, ml_model_values)

# Calculate RMSE
rmse = mean_squared_error(numerical_values, ml_model_values, squared=False)

# Calculate R2
r2 = r2_score(numerical_values, ml_model_values)

# Print the results
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Coefficient of Determination (R^2): {r2:.4f}")

Mean Absolute Error (MAE): 22.3883
Root Mean Squared Error (RMSE): 30.0021
Coefficient of Determination (R^2): -1.3233
